Author:
        
        PARK, JunHo, junho@ccnets.org

        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

# Credit Card Fraud Detection: Handling Imbalanced Dataset with CCNet

## Introduction

This tutorial explores the use of a Cooperative Network (CCNet) to address challenges associated with imbalanced datasets in the domain of credit card fraud detection. By leveraging the power of data generation, we aim to enhance the diversity and volume of training data, thereby improving the robustness and accuracy of models designed to identify fraudulent transactions.

## Tutorial Goals

The objectives of this tutorial are designed to guide you through the process of enhancing data quality and model performance:

### Dataset Recreation with CCNet
- **Understand Data Augmentation**: Learn how encoding techniques can be used to generate synthetic data instances that closely mimic the characteristics of real-world fraudulent and non-fraudulent transactions.
- **Impact on Model Training**: Assess how augmenting the dataset influences the training process and subsequently, the model's ability to generalize from training to real-world scenarios.

### Model Training and Evaluation
- **Dual Model Training**: Train two distinct models to directly compare performance metrics:
  - A model trained on the **original dataset**.
  - A model trained on the **CCNet-augmented dataset**.
- **Performance Metrics**: Use the F1 score, a critical measure for models operating on imbalanced datasets, to evaluate and compare the effectiveness of these models.

### Testing and Validation
- **Independent Model Testing**: Conduct a thorough evaluation of both models using a standalone test set that was not involved in the training phase.
- **Objective Analysis**: Critically analyze the outcomes to validate whether data augmentation through CCNet offers a tangible benefit in detecting credit card fraud.

## Conclusion

By the end of this tutorial, participants will not only grasp the theoretical underpinnings of using synthetic data to combat data imbalance but also gain hands-on experience in applying these concepts through CCNet to potentially enhance model performance in fraud detection tasks.


In [1]:
import sys
path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.


In [2]:
# https://fraud-detection-handbook.github.io/fraud-detection-handbook/Chapter_7_DeepLearning/FeedForwardNeuralNetworks.html
import pandas as pd 

dataroot = path_append + "../data/credit_card_fraud_detection/creditcard.csv"
df = pd.read_csv(dataroot)
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [3]:
print('No Frauds', round(df['Class'].value_counts()[0] / len(df) *100,2), '%of the dataset')
print('Frauds', round(df['Class'].value_counts()[1] / len(df) *100,2), '%of the dataset')

No Frauds 99.83 %of the dataset
Frauds 0.17 %of the dataset


In [4]:
import torch
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

# Standardizing the features (excluding the class column)
# Standardizing the features (excluding the class column)
sc = StandardScaler()
df.iloc[:, :-1] = sc.fit_transform(df.iloc[:, :-1])

n_features = len(df.iloc[:, :-1].columns)
n_classes = len(df.iloc[:, -1:].columns)

print(n_features, n_classes)

30 1


In [5]:

# Defining the labeled and unlabeled dataset classes
class LabeledDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        vals = torch.tensor(self.x[index], dtype=torch.float32)
        label = torch.tensor(self.y[index], dtype=torch.float32)
        return vals, label

#### Initial Setup and Model Configuration

This section initializes the environment by setting a fixed random seed to ensure reproducibility of results. It imports necessary configurations and initializes model parameters with specific configurations. The model specified here is set to have no core model but uses a 'deepfm' encoder model for data processing, which is particularly tailored for structured or tabular data like credit card transactions.


In [6]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into training and test sets for model evaluation
df_train, df_test = train_test_split(df, test_size=0.5, shuffle=False)
X_train, y_train = df_train.iloc[:, :-1].values, df_train.iloc[:, -1:].values
X_test, y_test = df_test.iloc[:, :-1].values, df_test.iloc[:, -1:].values

# Labeled datasets for supervised learning tasks
trainset = LabeledDataset(X_train, y_train)  # Corrected to include training data
testset = LabeledDataset(X_test, y_test)     # Test set with proper labels

# Printing the shapes of the datasets for verification
print(f"Labeled Trainset Shape: {len(trainset)}, {trainset.x.shape[1]}")
print(f"Labeled Testset Shape: {len(testset)}, {testset.x.shape[1]}")

Labeled Trainset Shape: 142403, 30
Labeled Testset Shape: 142404, 30


In [7]:
# Set a fixed random seed for reproducibility of experiments
from nn.utils.init import set_random_seed
set_random_seed(0)

# Importing configuration setups for ML parameters and data
import torch
from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from trainer_hub import TrainerHub

# Configuration for the data handling, defining dataset specifics and the task type
data_config = DataConfig(dataset_name='CreditCardFraudDetection', task_type='binary_classification', obs_shape=[n_features], label_size=n_classes)

# Initializing ML parameters without a core model and setting the encoder model to 'deepfm' with specific configurations
ml_params = MLParameters(core_model='deepfm', encoder_model='none')
ml_params.model.core_config.num_layers = 5
ml_params.model.core_config.dropout = 0.0
ml_params.model.core_config.d_model = 128

# Setting training parameters and device configuration
ml_params.training.num_epoch = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Create a TrainerHub instance to manage training and data processing
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False, use_full_eval=False)


#### Dataset Splitting for Training and Testing

The original dataset is split into training and testing parts to evaluate the model's performance accurately. This step is crucial for validating the effectiveness of the training on unseen data.


In [8]:
trainer_hub.train(trainset)

Epochs:   0%|          | 0/4 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

[0/4][50/2225][Time 1.20]
Unified LR across all optimizers: 0.0001995308238189185
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0415	Gen: 0.6193	Rec: 0.6230	E: 0.0379	R: 0.0452	P: 1.2008
[0/4][100/2225][Time 0.97]
Unified LR across all optimizers: 0.00019907191565870155
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0507	Gen: 0.5803	Rec: 0.5861	E: 0.0448	R: 0.0565	P: 1.1158
[0/4][150/2225][Time 0.96]
Unified LR across all optimizers: 0.00019861406295796434
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0658	Gen: 0.5480	Rec: 0.5595	E: 0.0543	R: 0.0773	P: 1.0416
[0/4][200/2225][Time 0.97]
Unified LR across all optimizers: 0.00019815726328921765
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0786	Gen: 0.4958	Rec: 0.5102	E: 0.0642	R: 0.0930	P: 0.9274
[0/4][250/2225][Time 0.96]
Unified LR across all optimizers: 0.00019770151423055492
--------------------Tr

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

[1/4][25/2225][Time 1.13]
Unified LR across all optimizers: 0.00018030592393534033
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0010	Gen: 0.2892	Rec: 0.2894	E: 0.0007	R: 0.0012	P: 0.5777
[1/4][75/2225][Time 0.98]
Unified LR across all optimizers: 0.0001798912318178735
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0011	Gen: 0.2764	Rec: 0.2766	E: 0.0009	R: 0.0013	P: 0.5518
[1/4][125/2225][Time 0.97]
Unified LR across all optimizers: 0.00017947749346581006
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0010	Gen: 0.2685	Rec: 0.2688	E: 0.0007	R: 0.0012	P: 0.5364
[1/4][175/2225][Time 1.05]
Unified LR across all optimizers: 0.0001790647066855505
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0007	Gen: 0.2806	Rec: 0.2807	E: 0.0006	R: 0.0008	P: 0.5606
[1/4][225/2225][Time 1.05]
Unified LR across all optimizers: 0.00017865286928854052
--------------------Trai

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

[2/4][0/2225][Time 0.99]
Unified LR across all optimizers: 0.00016293335327318117
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0012	Gen: 0.2541	Rec: 0.2545	E: 0.0008	R: 0.0017	P: 0.5074
[2/4][50/2225][Time 0.97]
Unified LR across all optimizers: 0.00016255861695947546
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0010	Gen: 0.2433	Rec: 0.2434	E: 0.0008	R: 0.0011	P: 0.4858
[2/4][100/2225][Time 1.14]
Unified LR across all optimizers: 0.00016218474251537463
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0011	Gen: 0.2447	Rec: 0.2451	E: 0.0007	R: 0.0015	P: 0.4887
[2/4][150/2225][Time 0.96]
Unified LR across all optimizers: 0.00016181172795863357
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0009	Gen: 0.2439	Rec: 0.2439	E: 0.0009	R: 0.0009	P: 0.4869
[2/4][200/2225][Time 1.00]
Unified LR across all optimizers: 0.0001614395713115662
--------------------Trai

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

[3/4][25/2225][Time 1.06]
Unified LR across all optimizers: 0.00014689600866445298
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0007	Gen: 0.2361	Rec: 0.2363	E: 0.0006	R: 0.0009	P: 0.4716
[3/4][75/2225][Time 0.94]
Unified LR across all optimizers: 0.00014655815721980301
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0007	Gen: 0.2299	Rec: 0.2301	E: 0.0005	R: 0.0010	P: 0.4592
[3/4][125/2225][Time 0.96]
Unified LR across all optimizers: 0.00014622108281191326
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0015	Gen: 0.2376	Rec: 0.2381	E: 0.0010	R: 0.0020	P: 0.4742
[3/4][175/2225][Time 0.96]
Unified LR across all optimizers: 0.00014588478365364866
--------------------Training Metrics--------------------
Trainer:  deepfm
Inf: 0.0013	Gen: 0.2427	Rec: 0.2430	E: 0.0010	R: 0.0015	P: 0.4844
[3/4][225/2225][Time 0.95]
Unified LR across all optimizers: 0.0001455492579619846
--------------------Tra

#### Data Loading and Synthetic Data Generation

This section deals with loading the unlabelled dataset, processing it through the trained model to create synthetic data. This data augmentation step is crucial for models that benefit from larger datasets, such as in fraud detection scenarios.


In [9]:
import torch
from tools.loader import collate_fn

batch_size = 64
len_trainset = len(trainset)
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

# Initialize the recreated dataset container
recreated_training_data = None
recreated_labels = None

# Generate synthetic data through the model to augment the training dataset
with torch.no_grad():
    for data, _ in train_loader:
        data = data.to(device)
        # Generate a large amount of synthetic data
        explain = trainer_hub.core_ccnet.explain(data)
        
        generated_data, generated_condition = trainer_hub.core_ccnet.generate(explain)

        generated_data = generated_data.clone().detach().cpu()
        generated_condition = generated_condition.clone().detach().cpu()

        # Concatenate the processed batch to the main dataset
        if recreated_training_data is None:
            recreated_training_data = generated_data
            recreated_labels = generated_condition
        else:
            recreated_training_data = torch.cat([recreated_training_data, generated_data], dim=0)
            recreated_labels = torch.cat([recreated_labels, generated_condition], dim=0)
        
        # Ensure we do not exceed the original trainset's size
        if len(recreated_training_data) >= len_trainset:
            recreated_training_data = recreated_training_data[:len_trainset]
            recreated_labels = recreated_labels[:len_trainset]
            break

print(f"Recreated Training Data Shape: {recreated_training_data.shape}")
print(f"Recreated Labels Shape: {recreated_labels.shape}")

Recreated Training Data Shape: torch.Size([142403, 30])
Recreated Labels Shape: torch.Size([142403, 1])


#### Data Preparation for Model Training

After synthetic data generation, this section separates the data and labels for training purposes, preparing them for use in machine learning models to ensure proper supervision and evaluation.


In [10]:
import torch

# Assuming recreated_dataset is a PyTorch tensor already available in your context
ccnet_recreated_dataset = LabeledDataset(recreated_training_data, recreated_labels)

# Print the shapes of the datasets for verification
print(f"Labeled Original Trainset Shape: {len(trainset)}, {trainset.x.shape[1]}")
print(f"CCNet Recreated Dataset Shape: {len(ccnet_recreated_dataset)}, {ccnet_recreated_dataset.x.shape[1]}")

print(f"Labeled Original Testset Shape: {len(testset)}, {testset.x.shape[1]}")

# Retrieve number of features and classes from the recreated dataset
num_features = recreated_training_data.shape[1]
num_classes = recreated_labels.shape[1]

print(f"Number of Features: {num_features}")
print(f"Number of Classes: {num_classes}")

Labeled Original Trainset Shape: 142403, 30
CCNet Recreated Dataset Shape: 142403, 30
Labeled Original Testset Shape: 142404, 30
Number of Features: 30
Number of Classes: 1


In [11]:
class DNN(torch.nn.Module):
    def __init__(self, input_size, output_size, num_layers=4, hidden_size=256):
        super(DNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Create a list to hold all layers
        layers = []
        
        # Input layer
        layers.append(torch.nn.Linear(input_size, hidden_size))
        layers.append(torch.nn.ReLU())
        
        # Hidden layers
        for _ in range(num_layers - 2):
            layers.append(torch.nn.Linear(hidden_size, hidden_size))
            layers.append(torch.nn.ReLU())
        
        # Output layer
        layers.append(torch.nn.Linear(hidden_size, output_size))
        
        # Register all layers
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        return torch.sigmoid(x)

#### Training Supervised Models

This section outlines the process of training supervised learning models using both original and synthetic datasets. The `train_supervised_model` function is designed to iterate through the dataset, perform forward passes, compute loss, and update model weights using backpropagation.


In [12]:
def train_supervised_model(model, dataset):
    # Initialize the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)
    # Ensure reproducibility by resetting the random seed
    set_random_seed(0)
    # Create DataLoader for batch processing
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Training loop
    for epoch in range(4):  # Train for 2 epochs as an example
        for i, (data, label) in enumerate(trainloader):
            data = data.to(device)
            label = label.to(device)
            # Perform forward pass
            output = model(data)
            # Compute loss
            loss = torch.nn.functional.binary_cross_entropy(output, label)
            # Backward pass to compute gradients
            loss.backward()
            # Update weights
            optimizer.step()
            # Reset gradients
            optimizer.zero_grad()


#### Model Training Using Recreated and Original Datasets

Models are trained using both datasets generated through the Data Augmentation process and the original dataset. This comparison helps to determine the effectiveness of the synthetic data in improving model performance.


In [13]:
# Initialize and train a model on the recreated dataset
model_trained_on_recreated = DNN(input_size=num_features, output_size=num_classes).to(device)
train_supervised_model(model_trained_on_recreated, ccnet_recreated_dataset)

# Initialize and train a model on the original dataset
model_trained_on_original = DNN(input_size=num_features, output_size=num_classes).to(device)
train_supervised_model(model_trained_on_original, trainset)

C:\Users\user\AppData\Local\Temp\ipykernel_23864\1819989639.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vals = torch.tensor(self.x[index], dtype=torch.float32)
C:\Users\user\AppData\Local\Temp\ipykernel_23864\1819989639.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.y[index], dtype=torch.float32)


#### Evaluating Model Performance

After training, the models are evaluated using the F1 score, a harmonic mean of precision and recall, which is particularly useful in the context of imbalanced datasets like fraud detection. This step is critical for assessing the quality of the models trained on different types of data.


In [ ]:
from sklearn.metrics import f1_score

def get_f1_score(model, testset, batch_size=batch_size):
    model.eval()  # Set the model to evaluation mode
    y_true = []
    y_pred = []
    # DataLoader for testing
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

    # No gradient computation needed during inference
    with torch.no_grad():
        for data, label in test_loader:
            data = data.to(device)
            label = label.to(device)
            output = model(data)
            # Process output for binary classification
            predicted = (output.squeeze() > 0.5).long()
            y_true.extend(label.squeeze().long().cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute and return the F1 score
    score = f1_score(y_true, y_pred, average='binary')
    return score

# Calculate F1 scores for both models
f1_score_original = get_f1_score(model_trained_on_original, testset)
f1_score_recreated = get_f1_score(model_trained_on_recreated, testset)

# Output the results
print("F1 score of the supervised learning model trained on the original data: ", f1_score_original)
print("F1 score of the supervised learning model trained on the recreated data: ", f1_score_recreated)
